<a href="https://colab.research.google.com/github/Trading-com-Dados/tutoriais_publicos/blob/main/Sharpe_Ratio_Reels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---




**Fórmula do Sharpe Ratio**



<img src="https://latex.codecogs.com/svg.image?Sharpe&space;Ratio&space;=&space;\frac{Retorno&space;do&space;Portfolio&space;-&space;Ativo&space;Livre&space;de&space;Risco}{Risco&space;do&space;Portfolio}" title="https://latex.codecogs.com/svg.image?Sharpe Ratio = \frac{Retorno do Portfolio - Ativo Livre de Risco}{Risco do Portfolio}" />

---

In [ ]:
!pip install investpy

In [ ]:
!pip install nasdaq-data-link

In [5]:
import investpy
import numpy as np
import pandas as pd
import nasdaqdatalink

In [ ]:
nasdaqdatalink.read_key('SUA CHAVE AQUI')

In [ ]:
#Carteira composta de 4 ativos e alocação FIXA de 25% em cada um

ativos = ['ITSA4', 'PETR4', 'ABEV3', 'VALE3']
pesos = np.array([0.25, 0.25,0.25, 0.25])

In [ ]:
#Criar uma carteira vazia para popular com os dados

carteira_precos = pd.DataFrame()

In [ ]:
#Estrutura de repetição para popular a carteira

for i in ativos:
  df = investpy.get_stock_historical_data(stock=i, country='Brazil', from_date='11/08/2021', to_date='12/08/2022').Close
  df.rename(i,inplace=True)
  df.columns = [i]
  carteira_precos = pd.concat([carteira_precos, df], axis=1)
  carteira_precos.index.name="Date"
carteira_precos

,ITSA4,PETR4,ABEV3,VALE3
Date,,,,
2021-08-11,11.35,28.67,16.57,98.99
2021-08-12,11.15,29.10,16.58,98.92
2021-08-13,11.22,29.35,16.65,98.11
2021-08-16,11.12,28.64,16.40,98.56
2021-08-17,10.97,27.03,16.32,96.93
...,...,...,...,...
2022-08-08,9.13,36.63,14.80,68.58
2022-08-09,9.30,37.23,14.74,70.00
2022-08-10,9.35,37.11,15.16,70.05


In [ ]:
#Calcular os retornos

retorno_precos = carteira_precos.pct_change()

In [ ]:
#Multiplicar os retornos pelos preços e somar

retorno_carteira = (retorno_precos * pesos).sum(axis=1)

In [ ]:
retorno_carteira

Date
2021-08-11    0.000000
2021-08-12   -0.000682
2021-08-13    0.002726
2021-08-16   -0.010883
2021-08-17   -0.022780
                ...   
2022-08-08    0.021505
2022-08-09    0.012913
2022-08-10    0.007840
2022-08-11    0.001618
2022-08-12   -0.036047
Length: 252, dtype: float64

In [ ]:
retorno_portfolio = pd.DataFrame()
retorno_portfolio['Retornos'] = retorno_carteira

In [ ]:
#Média do retorno da carteira
retorno_carteira.mean()

-0.000291589377910992

Cálculo da Volatilidade


In [ ]:
#Calcular a matriz de covariancia

cov_matrix = retorno_precos.cov()

In [ ]:
vol_carteira = np.sqrt(np.dot(pesos.T, np.dot(cov_matrix,pesos)))

In [ ]:
vol_carteira = np.sqrt(np.dot(pesos.T, np.dot(cov_matrix,pesos)))

In [ ]:
vol_ano = vol_carteira*np.sqrt(245)

In [ ]:
vol_ano

0.2047768727542813

CDI Anualizado

In [ ]:
#Buscar o CDI Anualizado

cdi_ano = nasdaqdatalink.get('BCB/4392', start_date='2022-01-25')
cdi_ano

,Value
Date,
2022-01-31,9.15
2022-02-28,10.49
2022-03-31,11.15
2022-04-30,11.65
2022-05-31,12.51
2022-06-30,12.89
2022-07-31,13.15
2022-08-31,13.48


In [ ]:
cdi_ano.iloc[-1]

Value    13.48
Name: 2022-08-31 00:00:00, dtype: float64

In [ ]:
#Cálculo do Sharpe

#Para anualizar o retorno médio verificar quantos dias 

sharpe = ((retorno_carteira.mean()*274)-(cdi_ano.iloc[-1]/100))/vol_ano

In [ ]:
sharpe

Value   -1.048436
Name: 2022-08-31 00:00:00, dtype: float64